# FlowJo Batch Analysis

Run this notebook to process raw fcs file data in a batch to return a folder containing a summary csv file with frequencies and counts for all populations across all samples and a file for each fcs sample that shows its event's corresponding gates.

Contact juliejin3@berkeley.edu if you encounter any issues.

### Installs

In [1]:
!pip install flowkit pandas

### Imports
Loads Python libraries used in this program

In [2]:
import flowkit as fk
import flowio
import pandas as pd
import os
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Folder setup
Creates dedicated directories to store all processed and output data.

In [3]:
base_folder = '/content/drive/MyDrive/batch_processing'
os.makedirs(base_folder, exist_ok=True) # creates base folder if it doesn't exist already
print(f"Before moving on, place your template workspace file and sample with the template gating strategy in your google drive folder: {base_folder}")

Before moving on, place your template workspace file and sample with the template gating strategy in your google drive folder: /content/drive/MyDrive/batch_processing


In [4]:
# folder with all the raw fcs files
input_fcs_folder = os.path.join(base_folder, "fcs_files")
os.makedirs(input_fcs_folder, exist_ok=True) # creates input fcs folder if it doesn't exist already
print(f"Before moving on, place all your raw fcs files that you want to batch analyze in this folder just made: {input_fcs_folder}")

Before moving on, place all your raw fcs files that you want to batch analyze in this folder just made: /content/drive/MyDrive/batch_processing/fcs_files


In [5]:
# FlowJo workspace file with analysis template
wsp_file = os.path.join(base_folder, "0815Ratios.wsp") # edit this to be your actual wsp file name if needed
# sample with gating strategy template
template_sample_name = "merged_downsampled.fcs" # edit this to be the name of your sample containing the template gating strategy if needed
template_sample_file = os.path.join(base_folder, template_sample_name)

# main output folder for all results
results_output_folder = os.path.join(base_folder, "analysis_results")
os.makedirs(results_output_folder, exist_ok=True) # creates output folder if it doesn't exist already
# summary csv file with all analyzed fcs files
summary_output_file = os.path.join(results_output_folder, "_batch_analysis_summary.csv")

print(f"All results will be saved in this folder after runnning the pipeline: {results_output_folder}")

All results will be saved in this folder after runnning the pipeline: /content/drive/MyDrive/batch_processing/analysis_results


In [6]:
# put True if you would like to generate individual csv files for each sample showing the gates each event is contained in (takes about 5 minutes per file) and the summary csv file
# put False if you would just like the summary csv file only
output_per_event_files = False

In [7]:
batch_size = 5 # adjust this number based on your file sizes/stability
max_retries = 3

### Processing pipeline
- Loads the uploaded FlowJo workspace as a template and applies the defined gating strategy to a batch of fcs files
- Generates a summary csv file with with frequencies and counts for all populations across all samples and individual csv files for each sample, showing only the gates each event is contained in efficiently.

In [8]:
def run_batch_analysis(wsp_file, template_sample_file, input_fcs_folder, results_output_folder, summary_output_file, output_per_event_files, batch_size, max_retries):
    if not os.path.exists(wsp_file):
        print(f"Error: FlowJo workspace file not found at {wsp_file}")
        return

    if not os.path.exists(input_fcs_folder):
        print(f"Error: Input FCS folder not found at {input_fcs_folder}")
        return

    print("--- FlowJo Workspace Batch Processing Script ---")
    print(f"Loading FlowJo workspace from: {wsp_file}")

    try:
        # gets list of all fcs files in the input folder
        list_of_fcs_files = [
            os.path.join(input_fcs_folder, f) for f in os.listdir(input_fcs_folder) if f.lower().endswith('.fcs')
        ]

        if not list_of_fcs_files:
            print("No FCS files found in the input folder. Exiting.")
            return

        # creates a FlowKit session based on workspace
        wsp = fk.Workspace(wsp_file, ignore_missing_files=True)
        # gets the gating strategy from the specified template sample
        g_strat = wsp.get_gating_strategy(template_sample_name)

        if not g_strat:
            print(f"Error: Could not find gating strategy for sample '{template_sample_file}'.")
            return

        print(f"Successfully loaded gating strategy from sample '{template_sample_file}'.")

        summary_results = []
        all_gate_info = g_strat.get_gate_ids()

        for i in range(0, len(list_of_fcs_files), batch_size):
            batch = (i // batch_size) + 1
            batch_files = list_of_fcs_files[i:i+batch_size]

            for attempt in range(max_retries):
                try:
                    print(f"\nProcessing batch {batch} of {len(list_of_fcs_files) // batch_size + 1}... Attempt {attempt + 1}/{max_retries}")

                    session = fk.Session(gating_strategy=g_strat, fcs_samples=batch_files)

                    print(f"  Starting analysis for {len(batch_files)} samples in this batch.")
                    session.analyze_samples() # runs the analysis for all samples in the session
                    print(f"  Analysis completed for batch {batch}.")

                    # iterates through each file to process
                    for sample_id in session.get_sample_ids():
                        filename = sample_id # sample_id is the filename (for now)
                        print(f"\nExtracting results for: {filename} (internal ID: {sample_id})")
                        current_sample = session.get_sample(sample_id=sample_id)
                        df_raw_channels = current_sample.as_dataframe(source='raw')
                        num_events = len(df_raw_channels)

                        # extracts summary results (frequencies & count)
                        sample_results = {}
                        sample_results['Sample_Name'] = filename

                        gating_result = session.get_gating_results(sample_id=sample_id)

                        totalintensity_tuple = None
                        for col_tuple in df_raw_channels.columns:
                            if isinstance(col_tuple, tuple) and len(col_tuple) > 0 and col_tuple[0].lower() == 'totalintensity':
                                totalintensity_tuple = col_tuple
                                break
                            elif isinstance(col_tuple, str) and col_tuple.lower() == 'totalintensity':
                                totalintensity_tuple = col_tuple
                                break

                        if not totalintensity_tuple:
                            print(f"  Warning: 'totalintensity' channel not found in {filename}. Mean intensity will be 'N/A'.")

                        for gate_name, gate_path in all_gate_info:
                            try:
                                freq = gating_result.get_gate_relative_percent(
                                    gate_name=gate_name,
                                    gate_path=gate_path
                                )
                                count = gating_result.get_gate_count(
                                    gate_name=gate_name,
                                    gate_path=gate_path
                                )

                                sample_results[f'{gate_name}_Frequency'] = freq
                                sample_results[f'{gate_name}_Count'] = count

                                mean_intensity = 'N/A'
                                if totalintensity_tuple:
                                    try:
                                        membership_array = gating_result.get_gate_membership(
                                            gate_name=gate_name,
                                            gate_path=gate_path
                                        )

                                        gated_events = df_raw_channels[membership_array]

                                        if not gated_events.empty:
                                            mean_intensity = gated_events[totalintensity_tuple].mean()
                                        else:
                                            mean_intensity = '0.0' # mean is 0 if no events are in gate
                                    except Exception as mean_e:
                                        print(f"  Warning: Could not get mean intensity for population '{gate_name}' on 'totalintensity' in sample '{filename}': {mean_e}")

                                sample_results[f'{gate_name}_Mean_totalintensity'] = mean_intensity

                            except Exception as pop_e:
                                print(f"  Warning: Could not get results for population '{gate_name}'(Path: {gate_path}) in sample '{filename}': {pop_e}")
                                sample_results[f'{gate_name}_Frequency'] = 'N/A'
                                sample_results[f'{gate_name}_Count'] = 'N/A'

                        summary_results.append(sample_results)

                        if output_per_event_files:
                            # extracts per-event gate membership data optionally
                            print(f"  Extracting per-event data for {filename}")

                            raw_labels = [col[0] if isinstance(col, tuple) else col for col in df_raw_channels.columns]

                            x_tuple = None
                            y_tuple = None
                            raw_labels = []
                            raw_cols = []

                            for col_tuple in df_raw_channels.columns:
                                label = col_tuple[0] if isinstance(col_tuple, tuple) else col_tuple
                                if col_tuple[0].lower() == 'x':
                                    x_tuple = col_tuple
                                elif col_tuple[0].lower() == 'y':
                                    y_tuple = col_tuple
                                else:
                                    raw_labels.append(label)
                                    raw_cols.append(col_tuple)

                            if x_tuple and y_tuple:
                                event_labels = df_raw_channels.apply(lambda row: f"Event ({row[x_tuple]}, {row[y_tuple]})", axis=1)
                            else:
                                event_labels = [f"Event (N/A, N/A)" for i in range(num_events)]
                                print(f"  Warning: 'X' or 'Y' channel not found in {filename}. Using N/A for Event (X,Y) labels.")

                            df_event_label = pd.DataFrame({'Event Label': event_labels})

                            def format_raw_vals(row):
                                formatted_vals = [f"{label}: {row[val]}" for label, val in zip(raw_labels, raw_cols)]
                                return ', '.join(formatted_vals)

                            df_raw_vals_formatted = df_raw_channels.apply(format_raw_vals, axis=1)
                            df_raw_vals_formatted = pd.DataFrame({'Raw Values': df_raw_vals_formatted})

                            gate_list = []
                            gate_names = []

                            for gate_name, gate_path in all_gate_info:
                                try:
                                    membership_array = gating_result.get_gate_membership(
                                        gate_name=gate_name,
                                        gate_path=gate_path
                                    )
                                    gate_list.append(pd.Series(membership_array, name=gate_name))
                                    gate_names.append(gate_name)
                                except Exception as member_e:
                                    print(f"  Warning (Per-Event Membership): Could not get membership for gate '{gate_name}' in sample '{filename}': {member_e}")
                                    gate_list.append(pd.Series([False] * num_events, name=gate_name)) # creates a series of False if membership can't be retrieved
                                    gate_names.append(gate_name)

                            if gate_list:
                                df_all_memberships = pd.concat(gate_list, axis=1)
                            else:
                                df_all_memberships = pd.DataFrame(index=range(num_events))

                            df_all_memberships.columns = gate_names

                            if not df_all_memberships.empty:
                                stacked_filtered = df_all_memberships.stack()

                                true_memberships_index = stacked_filtered[stacked_filtered].index

                                gate_names_series = pd.Series(true_memberships_index.get_level_values(1), index=true_memberships_index.get_level_values(0))
                                list_of_gates_per_event_series = gate_names_series.groupby(level=0).apply(list)

                                list_of_gates_per_event = list_of_gates_per_event_series.reindex(range(num_events), fill_value=[]).tolist()
                            else:
                                list_of_gates_per_event = [[] for i in range(num_events)]

                            df_expanded_gates = pd.DataFrame(list_of_gates_per_event)

                            if not df_expanded_gates.empty:
                                df_expanded_gates.columns = [f'Gate_{i+1}' for i in range(df_expanded_gates.shape[1])]

                            df_per_event = pd.concat([df_event_label, df_raw_vals_formatted, df_expanded_gates], axis=1)

                            # output path for per-event csv file
                            per_event_filepath = os.path.join(results_output_folder, f'{filename}_gates_per_event.csv')

                            # prevents the default Pandas index and removes header
                            df_per_event.to_csv(per_event_filepath, index=False, header=False)
                            print(f"  Per-event data saved to: {per_event_filepath}")

                    print(f"\nBatch {batch} complete!")
                    break

                except Exception as e:
                    print(f"Error processing batch {batch} attempt {attempt + 1}): {e}")
                    if "Transport endpoint is not connected" in str(e) and attempt < max_retries - 1:
                        print(f"  Connection error detected. Retrying in 10 seconds...")
                        time.sleep(10)
                    else:
                        print(f"Fatal error detected. Skipping batch {batch}")
                        break

        # saves the summary to a single csv file
        if summary_results:
            results_df = pd.DataFrame(summary_results)
            results_df.to_csv(summary_output_file, index=False)
            print(f"\nBatch analysis complete! Results saved to: {summary_output_file}")
        else:
            print("\nNo FCS files were processed or no results were generated.")

    except Exception as e:
        print(f"An error occurred while running the analysis: {e}")

### Run batch analysis
Runs batch analysis on all your fcs files!

In [9]:
if __name__ == "__main__":
    run_batch_analysis(wsp_file, template_sample_file, input_fcs_folder, results_output_folder, summary_output_file, output_per_event_files, batch_size, max_retries)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

Extracting results for: d3p45_D02.fcs (internal ID: d3p45_D02.fcs)

Extracting results for: d3p45_E02.fcs (internal ID: d3p45_E02.fcs)

Extracting results for: d3p45_F02.fcs (internal ID: d3p45_F02.fcs)

Batch 358 complete!

Processing batch 359 of 670... Attempt 1/3
  Starting analysis for 5 samples in this batch.
  Analysis completed for batch 359.

Extracting results for: d3p45_B01.fcs (internal ID: d3p45_B01.fcs)

Extracting results for: d3p45_C01.fcs (internal ID: d3p45_C01.fcs)

Extracting results for: d3p45_D01.fcs (internal ID: d3p45_D01.fcs)

Extracting results for: d3p45_E01.fcs (internal ID: d3p45_E01.fcs)

Extracting results for: d3p45_F01.fcs (internal ID: d3p45_F01.fcs)

Batch 359 complete!

Processing batch 360 of 670... Attempt 1/3
  Starting analysis for 5 samples in this batch.
  Analysis completed for batch 360.

Extracting results for: d3p45_A01.fcs (internal ID: d3p45_A01.fcs)

Extracting results for: d3p46_A08.fcs (internal ID: